# UNSW-NB15 Random Forest Anomaly

In [39]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction import FeatureHasher

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## File System

In [40]:
DATA_DIR =  "./datasets/unsw/"
testing_fname = "UNSW_NB15_testing-set.csv"
training_fname = "UNSW_NB15_training-set.csv"

## Reading Files and Column Labeling

In [43]:
df1 = pd.read_csv(DATA_DIR + training_fname)
df2 =  pd.read_csv(DATA_DIR  + testing_fname)
df = df1.append(df2)


label_column = ['label']
categorical_columns = ['proto', 'service', 'state']
drop_columns = ['id', 'sttl', 'dttl', 'swin', 'dwin', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']
numeric_columns = list(set(df.columns) - set(label_column) - set(categorical_columns) - set(drop_columns))

### Train Test Split

In [44]:
from sklearn.model_selection import train_test_split
y = df['label'].tolist()
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [45]:
type(X_train)

pandas.core.frame.DataFrame

## Preprocessing

In [46]:
print(type(df['proto']))
len(df['proto'].unique())

<class 'pandas.core.series.Series'>


133

### Scaling

In [47]:
scaler = sklearn.preprocessing.MinMaxScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.fit_transform(X_test[numeric_columns])

/Users/jgraham/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jgraham/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/jgraham/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

### Categorical Columns Hash Trick

In [48]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html
def column_hash_trick(categorical_column, n_hash_features):
    """Turns categorical columns into hashed with hash trick.
    
    Args:
        categorical_column: Pandas Series.
        n_hash_features: Integer for number of hash features desired.
        
    Returns:
        scipy.sparse.csr.csr_matrix of hash features.
    
    """
    categories = categorical_column.unique()
    h = FeatureHasher(n_features=n_hash_features, input_type='string')
    return h.transform(categories)
    

In [49]:
# Add to dataframe
#pd.concat([df[['Genre', 'Publisher']], pd.DataFrame(hashed_features1),pd.DataFrame(hashed_features2)],
#axis=1)
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('hash_proto', FeatureHasher(5, input_type='string'), 'proto'),
                      ('hash_service', FeatureHasher(5, input_type='string'), 'service'),
                      ('hash_state', FeatureHasher(5, input_type='string'), 'state'),
                      ('numeric_cols', 'passthrough', numeric_columns),
                       ('dropped', 'drop', drop_columns),
                       ('label_drop', 'drop', 'label')])



In [50]:
X_train = ct.fit_transform(X_train)
X_test = ct.fit_transform(X_test)

In [51]:
result.shape

NameError: name 'result' is not defined

## Random Forest Training

In [52]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=800, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [53]:
y_pred_rf

array([1, 1, 0, ..., 1, 1, 0])

In [54]:
np.sum(y_test == y_pred_rf) / len(y_test)

0.8969248203169872

In [55]:
rnd_clf.score(X_test, y_test)

0.8969248203169872